In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button
from scipy.constants import speed_of_light
import ipywidgets as widgets
from IPython.display import display, clear_output

# Constants
SPEED_OF_LIGHT_KM = speed_of_light * 0.001  # km/s

# Set up the figure
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(10, 8))
plt.close(fig)  # Prevent duplicate plot output in Colab

# Initial parameters
n1_init = 1.0
n2_init = 1.5
theta_i_init = 45  # degrees

# Function to calculate refraction angle
def snells_law(n1, n2, theta_i):
    theta_i_rad = np.radians(theta_i)
    if n1 == 0 or n2 == 0:
        return 0
    sin_theta_r = (n1/n2) * np.sin(theta_i_rad)
    if abs(sin_theta_r) > 1:
        return None  # Total internal reflection
    return np.degrees(np.arcsin(sin_theta_r))

# Function to update the plot
def update_plot(n1, n2, theta_i):
    ax.clear()

    # Redraw interface and normal
    ax.axhline(0, color='cyan', linestyle='-', linewidth=2)
    ax.axvline(0, color='lime', linestyle='--', linewidth=2)

    # Calculate angles
    theta_r = snells_law(n1, n2, theta_i)
    theta_i_rad = np.radians(theta_i)

    # Incident ray
    x_inc = [-1.5 * np.cos(theta_i_rad), 0]
    y_inc = [1.5 * np.sin(theta_i_rad), 0]
    ax.plot(x_inc, y_inc, 'white', linewidth=2, label=f'Incident (θᵢ={theta_i}°)')
    ax.arrow(x_inc[0]/2, y_inc[0]/2,
             x_inc[0]/4, y_inc[0]/4,
             head_width=0.1, fc='orange', ec='orange')

    # Reflected ray
    x_refl = [0, 1.5 * np.cos(theta_i_rad)]
    y_refl = [0, 1.5 * np.sin(theta_i_rad)]
    ax.plot(x_refl, y_refl, 'white', linewidth=2, linestyle=':', label=f'Reflected (θᵣ={theta_i}°)')
    ax.arrow(x_refl[1]/2, y_refl[1]/2,
             x_refl[1]/4, y_refl[1]/4,
             head_width=0.1, fc='orange', ec='orange')

    # Refracted ray
    if theta_r is not None:
        theta_r_rad = np.radians(theta_r)
        x_refr = [0, 1.5 * np.cos(theta_r_rad)]
        y_refr = [0, -1.5 * np.sin(theta_r_rad)]
        ax.plot(x_refr, y_refr, 'white', linewidth=2, label=f'Refracted (θₜ={theta_r:.1f}°)')
        ax.arrow(x_refr[1]/2, y_refr[1]/2,
                 x_refr[1]/4, y_refr[1]/4,
                 head_width=0.1, fc='orange', ec='orange')
    else:
        ax.text(0.5, -1.8, "TOTAL INTERNAL REFLECTION",
                color='red', ha='center', fontsize=12, fontweight='bold')

    # Add labels
    ax.text(-1.8, 1.8, f"Medium 1\nn₁ = {n1}\nv = {SPEED_OF_LIGHT_KM/n1:.2f} km/s",
            color='cyan', bbox=dict(facecolor='black', alpha=0.7))
    ax.text(-1.8, -1.8, f"Medium 2\nn₂ = {n2}\nv = {SPEED_OF_LIGHT_KM/n2:.2f} km/s",
            color='cyan', bbox=dict(facecolor='black', alpha=0.7))

    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_aspect('equal')
    ax.set_title(f"Snell's Law: {n1}·sin({theta_i}°) = {n2}·sin({theta_r:.1f}°)"
                if theta_r else f"TIR at {theta_i}° (n₁={n1}, n₂={n2})",
                pad=20, fontsize=14)
    ax.legend(loc='upper right')

    # Redraw the figure
    display(fig)
    clear_output(wait=True)

# Create sliders
n1_slider = widgets.FloatSlider(value=n1_init, min=1.0, max=3.0, step=0.1, description='n₁:')
n2_slider = widgets.FloatSlider(value=n2_init, min=1.0, max=3.0, step=0.1, description='n₂:')
theta_slider = widgets.IntSlider(value=theta_i_init, min=0, max=89, step=1, description='θᵢ:')

# Create output area
ui = widgets.VBox([n1_slider, n2_slider, theta_slider])
out = widgets.interactive_output(update_plot, {
    'n1': n1_slider,
    'n2': n2_slider,
    'theta_i': theta_slider
})

# Display the widgets and plot
display(ui, out)


Output()